# Download ultralytics

In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 51802, done.
remote: Counting objects: 100% (928/928), done.
remote: Compressing objects: 100% (632/632), done.
remote: Total 51802 (delta 668), reused 300 (delta 296), pack-reused 50874 (from 4)
Receiving objects: 100% (51802/51802), 30.26 MiB | 26.23 MiB/s, done.
Resolving deltas: 100% (38000/38000), done.


# check hardware config

In [2]:
!nvidia-smi

Tue Mar 11 07:28:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%pwd

'/kaggle/working'

# install in developer mode

In [2]:
%cd ultralytics
!pip install -e '.[dev]'

/kaggle/working/ultralytics
Obtaining file:///kaggle/working/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 37.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.3/380.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.5/128.5 k

#check installations

In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.3/8062.4 GB disk)


# Baseline YOLOv8 Model
create a new file, name it as train.py and check the model info.

In [6]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8n.yaml')
    model.info()

Writing train.py


In [7]:
!python train.py

YOLOv8n summary: 129 layers, 3,157,200 parameters, 3,157,184 gradients, 8.9 GFLOPs


# Everything is file and ready to go
Let's download the dataset from Roboflow

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("bud_detection-np")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.1 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to bud_detection-np-1 in yolov8:: 100%|██████████| 6468/6468 [00:01<00:00, 6432.43it/s]


# update data.yaml file

In [5]:
%%writefile /kaggle/working/ultralytics/bud_detection-np-1/data.yaml
names:
- BUD
nc: 1
roboflow:
  license: CC BY 4.0
  project: bud_detection-np
  url: https://universe.roboflow.com/buddetection/bud_detection-np/dataset/1
  version: 1
  workspace: buddetection
test: /kaggle/working/ultralytics/bud_detection-np-1/test/images
train: /kaggle/working/ultralytics/bud_detection-np-1/train/images
val: /kaggle/working/ultralytics/bud_detection-np-1/valid/images


Overwriting /kaggle/working/ultralytics/bud_detection-np-1/data.yaml


In [6]:
%pwd

'/kaggle/working/ultralytics'

# !!!START TRAINING!!!

In [13]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=64, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [14]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.86 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

# Results for baseline YOLOv8 are obtained

# Collect the results

In [37]:
!zip -r file.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train2/ (stored 0%)
  adding: runs/detect/train2/weights/ (stored 0%)
  adding: runs/detect/train2/args.yaml (deflated 53%)
  adding: runs/detect/train4/ (stored 0%)
  adding: runs/detect/train4/weights/ (stored 0%)
  adding: runs/detect/train4/args.yaml (deflated 53%)
  adding: runs/detect/train5/ (stored 0%)
  adding: runs/detect/train5/weights/ (stored 0%)
  adding: runs/detect/train5/weights/best.pt (deflated 10%)
  adding: runs/detect/train5/weights/last.pt (deflated 10%)
  adding: runs/detect/train5/train_batch2.jpg (deflated 0%)
  adding: runs/detect/train5/confusion_matrix_normalized.png (deflated 31%)
  adding: runs/detect/train5/events.out.tfevents.1741320144.01a8971d848d.141.0 (deflated 87%)
  adding: runs/detect/train5/labels_correlogram.jpg (deflated 41%)
  adding: runs/detect/train5/train_batch1620.jpg (deflated 4%)
  adding: runs/detect/train5/train_batch0.jpg (deflated 1%)
  adding: run

In [38]:
!zip -r file.zip runs
from IPython.display import FileLink
FileLink(r'/kaggle/working/ultralytics/file.zip')

/kaggle/working/ultralytics/file.zip

# Train and get results for visDrone dataset(Baseline YOLOv8m model)

In [39]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="VisDrone.yaml", epochs=50, batch=32, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [40]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8.yaml, data=VisDrone.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf

In [20]:
!zip -r file.zip runs
from IPython.display import FileLink
FileLink(r'/kaggle/working/ultralytics/file1.zip')

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/PR_curve.png (deflated 22%)
  adding: runs/detect/train/train_batch2.jpg (deflated 0%)
  adding: runs/detect/train/R_curve.png (deflated 18%)
  adding: runs/detect/train/args.yaml (deflated 54%)
  adding: runs/detect/train/P_curve.png (deflated 18%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 5%)
  adding: runs/detect/train/train_batch4230.jpg (deflated 4%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/last.pt (deflated 10%)
  adding: runs/detect/train/weights/best.pt (deflated 10%)
  adding: runs/detect/train/train_batch4231.jpg (deflated 4%)
  adding: runs/detect/train/confusion_matrix.png (deflated 39%)
  adding: runs/detect/train/train_batch1.jpg (deflated 1%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 8%)
  adding: runs/detect/train/F1_curve.png (deflated 18%)
  adding: runs/detect/

/kaggle/working/ultralytics/file1.zip

# Comparing with YOLOv6n

In [12]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('/kaggle/working/ultralytics/ultralytics/cfg/models/v6/yolov6n.yaml')
    model.info()

Overwriting train.py


In [13]:
!python train.py

YOLOv6n summary: 121 layers, 4,500,080 parameters, 4,500,064 gradients, 13.1 GFLOPs


In [16]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('/kaggle/working/ultralytics/ultralytics/cfg/models/v6/yolov6n.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=32, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [17]:
!python train.py

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/ultralytics/ultralytics/cfg/models/v6/yolov6n.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

# Comparing with YOLOv9n

In [6]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('/kaggle/working/ultralytics/ultralytics/cfg/models/v9/yolov9t.yaml')
    model.info()

Writing train.py


In [7]:
!python train.py

YOLOv9t summary: 544 layers, 2,128,720 parameters, 2,128,704 gradients, 8.5 GFLOPs


In [9]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('/kaggle/working/ultralytics/ultralytics/cfg/models/v9/yolov9t.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=16, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [10]:
!python train.py

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/ultralytics/ultralytics/cfg/models/v9/yolov9t.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx